# NLP: Review Summary, Sentiment Analysis
Reviews Summarizer on reviews, where is review summarizer lecture? Tips and recommendations here. SUMY  
- (done) Clean contractions, Lemmatize??, Remove Stop words (think specific to reviewing airbnb)
- parts of speech (removing pronouns, San Francisco)
- Tokenize
- Vectorize
- StandardScale
- Dim Reduction
- Topic Model
- Clustering on reviews: try all the methods, Kmeans, DBScan, .....
- Sumy LSA
- Gensim summarizer()TextBlob



## Imports

In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import dask.dataframe as dd
import pandas as pd
import numpy as np
import math
import os
import pickle
import timeit
from collections import Counter

# Graphics
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='svg'
import seaborn as sns
sns.set(font_scale=1.2)
plt.style.use('seaborn')
# from mlxtend.plotting import plot_decision_regions
# from mpl_toolkits.mplot3d import Axes3D

In [11]:
from __future__ import print_function
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from operator import itemgetter
import contractions

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
# %run '/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/notebooks/helper_functions.py' 
os.chdir('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/notebooks')
import helper_functions
from helper_functions import clean_dolla, plot_bar, cluster_inertia

In [13]:
os.chdir('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/data')
with open('listings_sf_cleaned.pkl', 'rb') as file:
    listings = pickle.load(file)

print(listings.shape)
listings.head()

(6633, 95)


,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,bathrooms,bedrooms,beds,price,weekly_price,...,neighborhood_Potrero Hill,neighborhood_Presidio Heights,neighborhood_Russian Hill,neighborhood_Seacliff,neighborhood_South of Market,neighborhood_Twin Peaks,neighborhood_Visitacion Valley,neighborhood_West of Twin Peaks,neighborhood_Western Addition,host_in_same_neighborhood
0,1,1,1,1,1,1.0,1.0,2.0,170.0,1120.0,...,0,0,0,0,0,0,0,0,1,1
1,0,2,2,1,1,1.0,2.0,3.0,235.0,1600.0,...,0,0,0,0,0,0,0,0,0,1
2,0,10,10,1,1,4.0,1.0,1.0,65.0,485.0,...,0,0,0,0,0,0,0,0,0,1
3,0,10,10,1,1,4.0,1.0,1.0,65.0,490.0,...,0,0,0,0,0,0,0,0,0,1
4,0,2,2,1,1,1.5,2.0,2.0,675.0,0.0,...,0,0,0,0,0,0,0,0,1,1


In [82]:
reviews=pd.read_csv('SF/201808/reviews_201808.csv')
reviews.columns
drop_list = ['reviewer_id', 'reviewer_name']
print(reviews.shape)
[reviews.drop(x, inplace=True, axis=1) for x in drop_list]
reviews.head()

(278884, 6)


,listing_id,id,date,comments
0,958,5977,2009-07-23,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,Holly's place was great. It was exactly what I...


## Preprocess

#### Preview

In [30]:
ix=2000
print(reviews['comments'][ix])
print()
print(contractions.fix(reviews['comments'][ix]))

I had great time staying at Brenda's house. She is an exellent host, very friendly and helpful. The place was beautiful and the location perfect. I can really recommend staying with Brenda while you're in San Francisco. I only had very few days in SF but I would like to come back and stay at Brenda's place again. Thanks for everything!!

I had great time staying at Brenda's house. She is an exellent host, very friendly and helpful. The place was beautiful and the location perfect. I can really recommend staying with Brenda while you are in San Francisco. I only had very few days in SF but I would like to come back and stay at Brenda's place again. Thanks for everything!!


In [57]:
reviews.head()

,listing_id,id,date,comments
0,958,5977,2009-07-23,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,Holly's place was great. It was exactly what I...


#### Actual work

In [103]:
def clean_text(text):
    if (type(text)==float): # For missings
        return ""
    else:
        text = contractions.fix(text)
        lemmatize = WordNetLemmatizer()
        tokens = word_tokenize(text)
        clean_text = [lemmatize.lemmatize(token.lower().strip(), pos='v') for token in tokens]
        clean_text = [x for x in clean_text if x not in stop]
        return ' '.join(clean_text)

In [105]:
reviews_cleaned = [clean_text(r) for r in reviews['comments']]
print(len(reviews_cleaned))
reviews_cleaned[:1]

278884


['experience without doubt five star experience holly husband david consummate host ; friendly accomodating still honor privacy apartment charm layout full view access home garden location perfect full engagement city ; close mass transit walk proximity haight mission castro golden gate park wait next visit ted karen wingerd']

In [107]:
# !pwd
# import pickle
# with open('reviews_cleaned.pkl', 'wb') as file:
#     pickle.dump(reviews_cleaned, file)

/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/data


In [108]:
# Remove Stop Words
stop = stopwords.words('english')
print(len(stop))
punctuations=['.', ',', '(', ')', "!", "?", "'", '"', "<", ">"]
more=["/n", "/t", "'s"]
stop += punctuations
print(len(stop))
stop += more
print(len(stop))

179
189
192


In [109]:
# Tokenize
tokenized = [word_tokenize(i) for i in reviews_cleaned]
print(len(tokenized))

278884


In [153]:
len(tokenized[80])

7

In [154]:
# Vectorize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2), stop_words=stop, lowercase=True)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words=stop, lowercase=True)

cv = [count_vectorizer.fit_transform(i) for i in tokenized if len(i)>0]
# tfidf = [tfidf_vectorizer.fit_transform(i) for i in tokenized]

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
# StandardScale
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.preprocessing import StandardScaler